<a href="https://colab.research.google.com/github/tony-wade/reverse-engineering/blob/main/Custom_CRC8_16_Calculator%26Searching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intro
CRC計算原理為**求指定區塊之多項式餘式**, 在多數塊加密之中也常出現。

屬於一種Hash，著重在其偵錯能力與計算效率
，因此不必可逆、不強制需要以 irreducible polynomials作為分式

(但collision越少越好, 可以是injective function)。





## CRC8

In [ ]:
def calculate_crc8(input_string, poly_hex='07', initial_value='00', output_xor='00'):
    data = bytes.fromhex(input_string)
    poly = int(poly_hex, 16)
    crc = int(initial_value, 16)

    for byte in data:
        crc ^= byte
        for _ in range(8):
            if crc & 0x80:
                crc = ((crc << 1) ^ poly) & 0xFF
            else:
                crc = (crc << 1) & 0xFF

    crc ^= int(output_xor, 16)
    return f'{crc:02X}'

def find_crc8_configurations(input_string, target_value):
    """
    Find CRC8 configurations for a specific input string and target value.

    :param input_string: Fixed input hexadecimal string
    :param target_value: Desired CRC8 value
    :return: List of matching configurations
    """
    target_value = target_value.upper()
    matching_configs = []

    # Iterate through possible polynomials (1-255)
    for poly in range(1, 256):
        poly_hex = f'{poly:02X}'

        # Test with standard initial values 00 and FF
        for initial_value in ['00', 'FF']:
            # Iterate through output XOR values (0-255)
            for xor_out in range(256):
                xor_hex = f'{xor_out:02X}'

                # Calculate CRC for the current configuration
                crc = calculate_crc8(input_string, poly_hex, initial_value, xor_hex)

                # Check if the calculated CRC matches the target value
                if crc == target_value:
                    matching_configs.append({
                        'poly': poly_hex,
                        'initial': initial_value,
                        'xor_out': xor_hex,
                        'input_value': input_string,
                        'crc_value': crc
                    })

    return matching_configs


def find_common_crc8_configurations(inputs_and_targets):
    """
    Find CRC8 configurations that match multiple input-target pairs.

    :param inputs_and_targets: List of tuples (input_string, target_value)
    :return: List of matching configurations
    """
    matching_configs = []

    # Iterate through possible polynomials (1-255)
    for poly in range(1, 256):
        poly_hex = f'{poly:02X}'

        # Iterate through possible initial values
        for init_value in range(256):
            initial_value = f'{init_value:02X}'
            # Iterate through output XOR values (0-255)
            for xor_out in range(256):
                xor_hex = f'{xor_out:02X}'

                # Check if this configuration works for ALL input-target pairs
                if all(
                    calculate_crc8(input_string, poly_hex, initial_value, xor_hex) == target_value.upper()
                    for input_string, target_value in inputs_and_targets
                ):
                    config = {
                        'poly': poly_hex,
                        'initial': initial_value,
                        'xor_out': xor_hex,
                        'results': [
                            {
                                'input_value': input_string,
                                'target_value': target_value,
                                'crc_value': calculate_crc8(input_string, poly_hex, initial_value, xor_hex)
                            }
                            for input_string, target_value in inputs_and_targets
                        ]
                    }
                    matching_configs.append(config)

    return matching_configs

In [ ]:
if __name__ == '__main__':
  input_value = 'EC8251'
  result = calculate_crc8(input_value, poly_hex='07', initial_value='00', output_xor='00')
  print(result)

18


In [ ]:
if __name__ == '__main__':
    # List of (input,target CRC8 value)
    inputs_and_targets = [
        ('c0fb4abd09134a3a34', '29'),
        ('c0167528267ed07a8f', '07'),
        ('c05da40f46f206a36e', 'b7'),
    ]

    results = find_common_crc8_configurations(inputs_and_targets)

    if results:
        print(f"\nFound {len(results)} common configurations:")
        for config in results:
            print(f"  Polynomial: {config['poly']}")
            print(f"  Initial Value: {config['initial']}")
            print(f"  Output XOR Value: {config['xor_out']}")
            print(" ")
    else:
        print("No common configurations found")

No common configurations found


## CRC16

改為搜索 CRC16, 若有相同的輸出在特定位置則輸出設定

In [ ]:
def calculate_crc16(input_string, poly_hex='8005', initial_value='0000', output_xor='0000', rev_in=True, rev_out=True):
    data = bytes.fromhex(input_string) # must in hex format
    poly = int(poly_hex, 16)
    crc = int(initial_value, 16)

    for byte in data:
        if rev_in:
            byte = int('{:08b}'.format(byte)[::-1], 2)

        crc ^= byte << 8
        for _ in range(8):
            if crc & 0x8000:
                crc = ((crc << 1) ^ poly) & 0xFFFF
            else:
                crc = (crc << 1) & 0xFFFF

    if rev_out:
        crc = int('{:016b}'.format(crc)[::-1], 2)

    crc ^= int(output_xor, 16)
    return f'{crc:04X}'


def find_common_crc16_configurations(inputs_and_targets, poly_hex=None):
    matching_configs = []

    # Convert target values to binary masks
    masks = [bin(int(target, 16))[2:].zfill(16) for _, target in inputs_and_targets]

    # If no poly list provided, use default initial poly
    if poly_hex is None:
        poly_list = ['8005','C867','3D65','1021','0589','8BB7','A097',]  # Common CRC-16 polynomial

    for poly_hex in poly_list:
        for initial_value in ['0000', 'FFFF', '00FF', 'FF00']:
            for xor_out in ['0000', 'FFFF', '00FF', 'FF00']:
                for rev_in in [True, False]:
                    for rev_out in [True, False]:
                        # Process original inputs
                        outputs = [
                            bin(int(calculate_crc16(input_string, poly_hex, initial_value, xor_out, rev_in, rev_out), 16))[2:].zfill(16)
                            for input_string, _ in inputs_and_targets
                        ]

                        # Verify if all masks can be aligned with outputs
                        all_match = True
                        for mask, output in zip(masks, outputs):
                            mask_len = len(mask.lstrip('0'))  # Effective mask length
                            for i in range(len(output) - mask_len + 1):
                                if output[i:i + mask_len] == mask.lstrip('0'):
                                    break
                            else:  # No match found for this mask
                                all_match = False
                                break

                        if all_match:  # Store configuration if all masks align
                            config = {
                                'poly': poly_hex,
                                'initial': initial_value,
                                'xor_out': xor_out,
                                'rev_in': rev_in,
                                'rev_out': rev_out,
                            }
                            matching_configs.append(config)

    return matching_configs

In [ ]:
if __name__ == '__main__':
    inputs_and_targets = [
        ('c0fb4abd09134a3a34', '29'),
        ('c0167528267ed07a8f', '07'),
        ('c05da40f46f206a36e', 'b7'),
        ('c0a75d38373a969eca', 'ad'),
        ('c00b64b68ea22ab7f8', '02'),
        ('c0c35d49afb2b66a5e', '25'),
        ('c02cd9c8dcade70eeb', 'dd'),
        ('c0538a55fb4ed9d42e', '2f'),
        ('c099f7e9c65458ed03', 'ad'),

    ]

    results = find_common_crc16_configurations(inputs_and_targets)

    if results:
        print(f"\nFound {len(results)} common configurations:")
        for config in results:
            print(f"  Polynomial: {config['poly']}")
            print(f"  Initial Value: {config['initial']}")
            print(f"  Output XOR Value: {config['xor_out']}")
            print(f"  Reverse Input/Output: {config['rev_in'], config['rev_out']}" )
            print(" ")
    else:
        print("No common configurations found")


Found 2 common configurations:
  Polynomial: 1021
  Initial Value: FFFF
  Output XOR Value: FFFF
  Reverse Input/Output: (True, True)
 
  Polynomial: 1021
  Initial Value: FFFF
  Output XOR Value: 00FF
  Reverse Input/Output: (True, True)
 


In [ ]:
if __name__ == '__main__':
  input_string = '471B6D2E'
  result = calculate_crc16(input_string,
                  poly_hex='1021',
                  initial_value='FFFF',
                  output_xor='00FF',
                  rev_in=True,
                  rev_out=True
                  )
  print(result)

  result = calculate_crc16(input_string,
                  poly_hex='1021',
                  initial_value='FFFF',
                  output_xor='FFFF',
                  rev_in=True,
                  rev_out=True
                  )
  print(result)

F32A
0C2A
